ref: https://zenn.dev/sinchir0/articles/9228eccebfbf579bfdf4

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.metrics import average_precision_score, mean_squared_error, mean_squared_log_error
import swifter
from pytz import timezone
import jpholiday
from datetime import datetime
import joblib
import swifter

pd.options.display.max_columns = 100

from tqdm.notebook import tqdm

import os
from logzero import logger
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import models, transforms
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, QuantileTransformer

from pytorch_tabnet.pretraining import TabNetPretrainer

from pytorch_tabnet.tab_model import TabNetRegressor
from utils import StratifiedGroupKFold

In [2]:
train, test, groups = joblib.load('../output/features.joblib')

In [3]:
X_train = train.drop(['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis=1)
y_train = np.log1p(train['Global_Sales'])
X_test = test.drop(['NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales', 'Global_Sales'], axis=1).copy()

In [4]:
stratified_y = pd.cut(y_train, 10, labels=False)

In [5]:
category_cols = X_train.columns[X_train.dtypes=='category']
for col in category_cols:
    X_train[col] = X_train[col].astype(object)
    X_test[col] = X_test[col].astype(object)

object_cols = X_train.columns[X_train.dtypes=='object']
for col in tqdm(object_cols):
    X_train[col] = X_train[col].astype(object).astype(str)
    X_test[col] = X_test[col].astype(object).astype(str)
    le = LabelEncoder()
    le.fit(X_train[col])
    le_dict = dict(zip(le.classes_, le.transform(le.classes_)))
    X_train[col] = X_train[col].swifter.apply(lambda x: le_dict.get(x, np.nan))
    X_test[col] = X_test[col].swifter.apply(lambda x: le_dict.get(x, np.nan))

In [6]:
X_train = X_train.fillna(-1)
X_test = X_test.fillna(-1)

In [7]:
SEED = 7188

In [8]:
feature_col = X_train.columns.tolist()
categorical_columns = object_cols

# class num
categorical_dims = {}

for col in categorical_columns:
    categorical_dims[col] = train[col].nunique()
cat_idxs = [ i for i, f in enumerate(feature_col) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(feature_col) if f in categorical_columns]

In [9]:
tabnet_params = dict(n_d=8, n_a=8, n_steps=3, gamma=1.3,
                     n_independent=2, n_shared=2,
                     seed=SEED, lambda_sparse=1e-3, 
                     optimizer_fn=torch.optim.Adam, 
                     optimizer_params=dict(lr=2e-2),
                     mask_type="entmax",
                     scheduler_params=dict(mode="min",
                                           patience=5,
                                           min_lr=1e-5,
                                           factor=0.9,),
                     scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                     verbose=10
                    )

pretrainer = TabNetPretrainer(**tabnet_params)

pretrainer.fit(
    X_train=X_train.values,
    eval_set=[X_train.values],
    max_epochs=200,
    patience=20, batch_size=256, virtual_batch_size=128,
    num_workers=1, drop_last=True)

Device used : cuda
epoch 0  | loss: 3425758374.62069| val_0_unsup_loss: 4446264320.0|  0:00:01s
epoch 10 | loss: 34317847.53448| val_0_unsup_loss: 277811776.0|  0:00:13s
epoch 20 | loss: 9543188.74138| val_0_unsup_loss: 157981760.0|  0:00:26s
epoch 30 | loss: 12542238.82759| val_0_unsup_loss: 115461648.0|  0:00:39s
epoch 40 | loss: 9429415.84483| val_0_unsup_loss: 51520564.0|  0:00:52s
epoch 50 | loss: 8119439.94828| val_0_unsup_loss: 93479200.0|  0:01:05s
epoch 60 | loss: 40494455.82759| val_0_unsup_loss: 54389300.0|  0:01:18s
epoch 70 | loss: 6256787.53448| val_0_unsup_loss: 206014080.0|  0:01:30s
epoch 80 | loss: 4626535.33621| val_0_unsup_loss: 91271680.0|  0:01:43s
epoch 90 | loss: 79153489.30172| val_0_unsup_loss: 120055312.0|  0:01:56s

Early stopping occured at epoch 97 with best_epoch = 77 and best_val_0_unsup_loss = 37904580.0
Best weights from best epoch are automatically used!


In [10]:
#kf = GroupKFold(n_splits=5)
kf = StratifiedGroupKFold(n_splits=5, random_state=0)
oof = np.zeros(X_train.shape[0])
y_pred = np.zeros(X_test.shape[0])
models = []

for i, (train_index, valid_index) in enumerate(kf.split(X_train, y=stratified_y, groups=groups)):
    print(f'Start {i+1} fold')
    print('-' * 20)
    X_tr, y_tr = X_train.iloc[train_index, :], y_train.iloc[train_index]
    X_val, y_val = X_train.iloc[valid_index, :], y_train.iloc[valid_index]
    X_tes = X_test.copy()


    tabnet_params = dict(n_d=8, n_a=8, n_steps=3, gamma=1.3,
                         n_independent=2, n_shared=2,
                         seed=SEED, lambda_sparse=1e-3,
                         optimizer_fn=torch.optim.Adam,
                         optimizer_params=dict(lr=2e-2,
                                               weight_decay=1e-5
                                              ),
                         mask_type="entmax",
                         scheduler_params=dict(max_lr=0.05,
                                               steps_per_epoch=int(X_train.shape[0] / 256),
                                               epochs=200,
                                               is_batch_level=True
                                              ),
                         scheduler_fn=torch.optim.lr_scheduler.OneCycleLR,
                         verbose=10,
                         cat_idxs=cat_idxs, # comment out when Unsupervised
                         cat_dims=cat_dims, # comment out when Unsupervised
                         cat_emb_dim=1 # comment out when Unsupervised
                        )

    model = TabNetRegressor(**tabnet_params)

    model.fit(X_train=X_tr.values,
              y_train=y_tr.values.reshape(-1,1),
              eval_set=[(X_val.values, y_val.values.reshape(-1,1))],
              eval_name = ["valid"],
              eval_metric = ["rmse"],
              max_epochs=200,
              patience=20, batch_size=256, virtual_batch_size=128,
              num_workers=0, drop_last=False,
              from_unsupervised=pretrainer # comment out when Unsupervised
             )
    
    
    
    oof[valid_index] = model.predict(X_val.values).reshape(-1)
    y_pred += model.predict(X_tes.values).reshape(-1) / 5
    models.append(model)

oof = np.where(oof < 0, 0, oof)
y_pred = np.where(y_pred < 0, 0, y_pred)
oof = np.expm1(oof)
y_pred = np.expm1(y_pred)

Start 1 fold
--------------------
Device used : cuda
Loading weights from unsupervised pretraining


/home/nekoumei/anaconda3/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:93: UserWarning: Pretraining: cat_dims changed from [31, 12, 8, 272, 64, 104, 105, 94, 40] to []
  warnings.warn(wrn_msg)
/home/nekoumei/anaconda3/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:93: UserWarning: Pretraining: cat_idxs changed from [0, 2, 7, 13, 14, 15, 16, 17, 18] to []
  warnings.warn(wrn_msg)


epoch 0  | loss: 62.82089| valid_rmse: 6.78092 |  0:00:00s
epoch 10 | loss: 1.06742 | valid_rmse: 1.04796 |  0:00:06s
epoch 20 | loss: 0.83685 | valid_rmse: 1.00857 |  0:00:13s
epoch 30 | loss: 0.73854 | valid_rmse: 0.96095 |  0:00:19s
epoch 40 | loss: 0.66452 | valid_rmse: 0.98226 |  0:00:25s
epoch 50 | loss: 0.62495 | valid_rmse: 0.97792 |  0:00:31s
epoch 60 | loss: 0.56589 | valid_rmse: 0.93317 |  0:00:37s
epoch 70 | loss: 0.51662 | valid_rmse: 0.94141 |  0:00:43s
epoch 80 | loss: 0.42195 | valid_rmse: 0.9674  |  0:00:50s
epoch 90 | loss: 0.38841 | valid_rmse: 0.96514 |  0:00:56s

Early stopping occured at epoch 99 with best_epoch = 79 and best_valid_rmse = 0.92444
Best weights from best epoch are automatically used!
Start 2 fold
--------------------
Device used : cuda
Loading weights from unsupervised pretraining


/home/nekoumei/anaconda3/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:93: UserWarning: Pretraining: cat_dims changed from [31, 12, 8, 272, 64, 104, 105, 94, 40] to []
  warnings.warn(wrn_msg)
/home/nekoumei/anaconda3/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:93: UserWarning: Pretraining: cat_idxs changed from [0, 2, 7, 13, 14, 15, 16, 17, 18] to []
  warnings.warn(wrn_msg)


epoch 0  | loss: 62.80843| valid_rmse: 7.17968 |  0:00:00s
epoch 10 | loss: 1.12279 | valid_rmse: 1.07252 |  0:00:06s
epoch 20 | loss: 0.81682 | valid_rmse: 0.97995 |  0:00:13s
epoch 30 | loss: 0.77316 | valid_rmse: 0.95375 |  0:00:19s
epoch 40 | loss: 0.64452 | valid_rmse: 0.95526 |  0:00:25s
epoch 50 | loss: 0.59384 | valid_rmse: 0.98991 |  0:00:31s
epoch 60 | loss: 0.55006 | valid_rmse: 0.9595  |  0:00:38s

Early stopping occured at epoch 68 with best_epoch = 48 and best_valid_rmse = 0.93898
Best weights from best epoch are automatically used!
Start 3 fold
--------------------
Device used : cuda
Loading weights from unsupervised pretraining


...

epoch 0  | loss: 62.79309| valid_rmse: 6.9796  |  0:00:00s
epoch 10 | loss: 1.13787 | valid_rmse: 1.08276 |  0:00:06s
epoch 20 | loss: 0.80746 | valid_rmse: 0.98717 |  0:00:13s
epoch 30 | loss: 0.79034 | valid_rmse: 0.99094 |  0:00:19s
epoch 40 | loss: 0.74162 | valid_rmse: 1.03515 |  0:00:25s
epoch 50 | loss: 0.64123 | valid_rmse: 0.94233 |  0:00:31s
epoch 60 | loss: 0.58512 | valid_rmse: 0.9853  |  0:00:37s
epoch 70 | loss: 0.53799 | valid_rmse: 0.97456 |  0:00:44s

Early stopping occured at epoch 77 with best_epoch = 57 and best_valid_rmse = 0.93701
Best weights from best epoch are automatically used!
Start 4 fold
--------------------
Device used : cuda
Loading weights from unsupervised pretraining


/home/nekoumei/anaconda3/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:93: UserWarning: Pretraining: cat_dims changed from [31, 12, 8, 272, 64, 104, 105, 94, 40] to []
  warnings.warn(wrn_msg)
/home/nekoumei/anaconda3/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:93: UserWarning: Pretraining: cat_idxs changed from [0, 2, 7, 13, 14, 15, 16, 17, 18] to []
  warnings.warn(wrn_msg)


epoch 0  | loss: 62.76523| valid_rmse: 6.87865 |  0:00:00s
epoch 10 | loss: 1.10592 | valid_rmse: 1.12535 |  0:00:06s
epoch 20 | loss: 0.82155 | valid_rmse: 1.00189 |  0:00:13s
epoch 30 | loss: 0.70872 | valid_rmse: 1.00645 |  0:00:19s
epoch 40 | loss: 0.61201 | valid_rmse: 0.94757 |  0:00:25s
epoch 50 | loss: 0.64011 | valid_rmse: 0.96384 |  0:00:31s
epoch 60 | loss: 0.55526 | valid_rmse: 0.9946  |  0:00:37s

Early stopping occured at epoch 61 with best_epoch = 41 and best_valid_rmse = 0.93673
Best weights from best epoch are automatically used!
Start 5 fold
--------------------
Device used : cuda
Loading weights from unsupervised pretraining


/home/nekoumei/anaconda3/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:93: UserWarning: Pretraining: cat_dims changed from [31, 12, 8, 272, 64, 104, 105, 94, 40] to []
  warnings.warn(wrn_msg)
/home/nekoumei/anaconda3/lib/python3.7/site-packages/pytorch_tabnet/abstract_model.py:93: UserWarning: Pretraining: cat_idxs changed from [0, 2, 7, 13, 14, 15, 16, 17, 18] to []
  warnings.warn(wrn_msg)


epoch 0  | loss: 62.63964| valid_rmse: 6.7966  |  0:00:00s
epoch 10 | loss: 1.14947 | valid_rmse: 1.05711 |  0:00:06s
epoch 20 | loss: 0.82514 | valid_rmse: 0.93086 |  0:00:13s
epoch 30 | loss: 0.79037 | valid_rmse: 0.94537 |  0:00:19s
epoch 40 | loss: 0.65116 | valid_rmse: 0.91686 |  0:00:25s

Early stopping occured at epoch 43 with best_epoch = 23 and best_valid_rmse = 0.8975
Best weights from best epoch are automatically used!


In [11]:
y_train = np.expm1(y_train)
print(mean_squared_log_error(y_train, oof) ** .5)

0.927090275353453


In [12]:
joblib.dump((oof, y_pred), '../output/TabNet_oof_preds.joblib')

['../output/TabNet_oof_preds.joblib']